# POC of Backend Flow for Pokeradar

## Imports

In [42]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

In [43]:
def initialize_driver():
    # Initialize the driver (ensure ChromeDriver is in your system's PATH)
    driver = webdriver.Chrome()
    return driver

def click_state_link(driver, state):
    # Open the page with the state as a parameter
    url = "https://support.pokemoncenter.com/hc/en-us/sections/13360842288916-Pok%C3%A9mon-Automated-Retail-Vending-Machines"
    driver.get(url)
    
    # Wait for the page to load
    time.sleep(3)
    
    # Find the link based on the state (handle both full state name and abbreviation)
    state_link_xpath = f"//a[contains(text(), '{state} Pokémon Automated Retail Vending Machine Locations')]"
    
    try:
        # Locate and click the link
        state_link = driver.find_element(By.XPATH, state_link_xpath)
        state_link.click()
        print(f"Clicked on {state} link")
    except Exception as e:
        print(f"Error: {e}")
        print(f"Could not find link for {state}")

    # Wait for the redirected page to load
    time.sleep(3)

def extract_table(driver):
    # Find the table element
    table = driver.find_element(By.XPATH, "//table")
    
    # Get all rows from the table
    rows = table.find_elements(By.TAG_NAME, "tr")
    
    # Extract table headers
    headers = [header.text for header in rows[0].find_elements(By.TAG_NAME, "th")]
    
    # Extract table data
    table_data = []
    for row in rows[1:]:  # Skip the header row
        cells = row.find_elements(By.TAG_NAME, "td")
        row_data = [cell.text for cell in cells]
        table_data.append(row_data)
    
    # Convert to DataFrame using Pandas
    df = pd.DataFrame(table_data, columns=headers)
    
    return df


In [44]:
state = "California"

In [45]:
driver = initialize_driver()
click_state_link(driver, state)
df = extract_table(driver)
time.sleep(5)

Clicked on California link


In [46]:
# Close the browser after completion
driver.quit()

In [47]:
df

,Retailer,Machine ID,Address,"City, State"
0,Safeway,Q01036,2227 S Shore Center,"Alameda, CA"
1,Safeway,Q01268,2600 5th St,"Alameda, CA"
2,Food 4 Less,Q00330,1616 W Katella Ave,"Anaheim, CA"
3,Albertsons,Q00491,810 S State College Blvd,"Anaheim, CA"
4,Vons,Q00695,5600 Santa Ana Canyon Rd,"Anaheim, CA"
...,...,...,...,...
239,Pavilions,Q00415,8969 Santa Monica Blvd,"West Hollywood, CA"
240,Albertsons,Q00662,6755 Westminster Blvd,"Westminster, CA"
241,Vons,Q00327,15740 La Forge St,"Whittier, CA"
242,Vons,Q01240,20445 Yorba Linda Blvd,"Yorba Linda, CA"


In [48]:
df_imputed = df.copy()
df_imputed['Address'] = df_imputed.apply(lambda row: row['Address'] + ', ' + row['City, State'], axis=1)
df_imputed[['City', 'State']] = df_imputed['City, State'].apply(lambda x: pd.Series(x.split(', ')))
df_imputed.drop(columns=['City, State'], inplace=True)
df_imputed

,Retailer,Machine ID,Address,City,State
0,Safeway,Q01036,"2227 S Shore Center, Alameda, CA",Alameda,CA
1,Safeway,Q01268,"2600 5th St, Alameda, CA",Alameda,CA
2,Food 4 Less,Q00330,"1616 W Katella Ave, Anaheim, CA",Anaheim,CA
3,Albertsons,Q00491,"810 S State College Blvd, Anaheim, CA",Anaheim,CA
4,Vons,Q00695,"5600 Santa Ana Canyon Rd, Anaheim, CA",Anaheim,CA
...,...,...,...,...,...
239,Pavilions,Q00415,"8969 Santa Monica Blvd, West Hollywood, CA",West Hollywood,CA
240,Albertsons,Q00662,"6755 Westminster Blvd, Westminster, CA",Westminster,CA
241,Vons,Q00327,"15740 La Forge St, Whittier, CA",Whittier,CA
242,Vons,Q01240,"20445 Yorba Linda Blvd, Yorba Linda, CA",Yorba Linda,CA


In [49]:
df_imputed.to_csv('../data/data.csv')